<a href="https://colab.research.google.com/github/Sounakray2003/RLHF/blob/main/SFT%2BDPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install trl
!pip install -U bitsandbytes>=0.46.1
!pip install flash_attn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 530.9/530.9 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 55.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash_attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=253780426 sha256=4e2f9e39313266b1544b68138b15b91ee6221eccf14f7902b7c6620351340810
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully built flash_attn


In [2]:
import os
import json
import re
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive')
import torch
from PIL import Image
from tqdm import tqdm
from transformers import BlipProcessor, BlipForConditionalGeneration

# ─────────────────────────────
# CONFIG
# ─────────────────────────────
IMAGE_DIR = r"/content/drive/MyDrive/data/images"
SFT_OUT = "/content/drive/MyDrive/data/sft/train.json"
DPO_OUT = "/content/drive/MyDrive/data/dpo/train.json"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
VALID_EXT = (".jpg", ".jpeg", ".png", ".webp")

# ─────────────────────────────
# CREATE DIRS
# ─────────────────────────────
os.makedirs("data/sft", exist_ok=True)
os.makedirs("data/dpo", exist_ok=True)

# ─────────────────────────────
# LOAD BLIP
# ─────────────────────────────
print("Loading BLIP model...")
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained(
    "Salesforce/blip-image-captioning-base"
).to(DEVICE)

# ─────────────────────────────
# HELPERS
# ─────────────────────────────
def clean_caption(text: str) -> str:
    text = re.sub(r"\b(\w+)( \1\b)+", r"\1", text)  # remove repetition
    text = re.sub(r"\s+", " ", text)
    return text.strip()

def generate_caption(img_path: Path) -> str:
    image = Image.open(img_path).convert("RGB")
    inputs = processor(image, return_tensors="pt").to(DEVICE)
    output = model.generate(**inputs, max_new_tokens=30)
    caption = processor.decode(output[0], skip_special_tokens=True)
    return clean_caption(caption)

def strong_description(caption: str) -> str:
    return (
        f"{caption.capitalize()}. "
        "Clean visual appearance with good composition and clarity. "
        "Well-suited for professional listings and marketing use."
    )

def weak_description(caption: str) -> str:
    return (
        f"{caption.capitalize()}. "
        "Basic description with limited detail and no promotional context."
    )

# ─────────────────────────────
# MAIN LOOP
# ─────────────────────────────
images = [
    p for p in Path(IMAGE_DIR).iterdir()
    if p.suffix.lower() in VALID_EXT
]

print(f"Found {len(images)} images")

with open(SFT_OUT, "w", encoding="utf-8") as sft_f, \
     open(DPO_OUT, "w", encoding="utf-8") as dpo_f:

    for img in tqdm(images, desc="Building dataset"):
        caption = generate_caption(img)
        rel_path = f"images/{img.name}"

        chosen = strong_description(caption)
        rejected = weak_description(caption)

        # ───── SFT ─────
        sft_sample = {
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "Create a short, SEO-friendly product description based on the image."},
                        {"type": "image", "image": rel_path}
                    ]
                },
                {
                    "role": "assistant",
                    "content": [
                        {"type": "text", "text": chosen}
                    ]
                }
            ]
        }
        sft_f.write(json.dumps(sft_sample) + "\n")

        # ───── DPO ─────
        dpo_sample = {
            "prompt": [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "Create a short, SEO-friendly product description based on the image."},
                        {"type": "image", "image": rel_path}
                    ]
                }
            ],
            "chosen": chosen,
            "rejected": rejected
        }
        dpo_f.write(json.dumps(dpo_sample) + "\n")

print("✅ Clean SFT and DPO datasets created")
print(f"→ {SFT_OUT}")
print(f"→ {DPO_OUT}")


Mounted at /content/drive
Loading BLIP model...


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

The image processor of type `BlipImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. 


config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

...

The tied weights mapping and config for this model specifies to tie text_decoder.cls.predictions.bias to text_decoder.cls.predictions.decoder.bias, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie text_decoder.bert.embeddings.word_embeddings.weight to text_decoder.cls.predictions.decoder.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
BlipForConditionalGeneration LOAD REPORT from: Salesforce/blip-image-captioning-base
Key                                       | Status     |  | 
------------------------------------------+------------+--+-
text_decoder.bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identic

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Found 50 images



Building dataset: 100%|██████████| 50/50 [00:57<00:00,  1.15s/it]

✅ Clean SFT and DPO datasets created
→ /content/drive/MyDrive/data/sft/train.json
→ /content/drive/MyDrive/data/dpo/train.json


TRAINING

In [3]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoProcessor,
    TrainingArguments,
    Trainer,
    Qwen2VLForConditionalGeneration,
    BitsAndBytesConfig # Re-adding BitsAndBytesConfig from transformers
)
from peft import LoraConfig, get_peft_model
from PIL import Image

# ============================================================
# GPU + TORCH SETTINGS (VERY IMPORTANT)
# ============================================================
assert torch.cuda.is_available(), "❌ GPU not available"

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

DEVICE = "cuda"
DTYPE = torch.bfloat16

print("✅ GPU:", torch.cuda.get_device_name(0))

# ============================================================
# CONFIG
# ============================================================
MODEL_ID = "Qwen/Qwen2-VL-2B-Instruct"

IMAGE_ROOT = "/content/drive/MyDrive/data"
SFT_DATA = "/content/drive/MyDrive/data/sft/train.json"

OUTPUT_DIR = "outputs/qwen2_vl_lora_15gb"

# ============================================================
# LOAD PROCESSOR
# ============================================================
quantization_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
processor = AutoProcessor.from_pretrained(
    MODEL_ID,
    trust_remote_code=True,
    quantization_config=quantization_config
)

# ============================================================
# LOAD MODEL
# ============================================================
model = Qwen2VLForConditionalGeneration.from_pretrained(
    MODEL_ID,
    torch_dtype=DTYPE,
    device_map="auto",
    trust_remote_code=True
)

# ============================================================
# 🔥 FREEZE VISION ENCODER (CRITICAL FOR 15GB)
# ============================================================
for param in model.model.visual.parameters():
    param.requires_grad = False

# ============================================================
# APPLY LORA (LOW-RANK, LOW-VRAM)
# ============================================================
lora_config = LoraConfig(
    r=4,                     # 🔥 VERY IMPORTANT
    lora_alpha=8,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
model.train()

# ============================================================
# IMAGE LOADER (DOWNSCALE TO SAVE VRAM)
# ============================================================
def load_image(rel_path: str):
    full_path = os.path.join(IMAGE_ROOT, rel_path)
    if not os.path.exists(full_path):
        raise FileNotFoundError(full_path)

    img = Image.open(full_path).convert("RGB")
    img = img.resize((224, 224))   # 🔥 HUGE MEMORY SAVER
    return img

# ============================================================
# SFT COLLATOR (CPU ONLY)
# ============================================================
def sft_collator(batch):
    texts, images = [], []

    for sample in batch:
        messages = sample["messages"]

        user_content = messages[0]["content"]
        instruction = next(x["text"] for x in user_content if x["type"] == "text")
        image_path = next(x["image"] for x in user_content if x["type"] == "image")
        answer = messages[1]["content"][0]["text"]

        chat = [
            {
                "role": "user",
                "content": [
                    {"type": "image"},
                    {"type": "text", "text": instruction},
                ],
            },
            {
                "role": "assistant",
                "content": answer,
            },
        ]

        texts.append(
            processor.apply_chat_template(
                chat,
                tokenize=False,
                add_generation_prompt=False,
            )
        )

        images.append(load_image(image_path))

    inputs = processor(
        text=texts,
        images=images,
        padding=True,
        return_tensors="pt",
    )

    inputs["labels"] = inputs["input_ids"].clone()
    return inputs

# ============================================================
# LOAD DATASET
# ============================================================
dataset = load_dataset("json", data_files=SFT_DATA, split="train")

# ============================================================
# TRAINING ARGS (15GB SAFE)
# ============================================================
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=1,        # 🔥 DO NOT INCREASE
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    learning_rate=2e-5,
    logging_steps=10,
    bf16=True,
    gradient_checkpointing=True,
    remove_unused_columns=False,
    report_to="none",
    dataloader_num_workers=2,
)

# ============================================================
# TRAIN
# ============================================================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=sft_collator,
)

print("\n🔥 Starting SFT (15GB GPU SAFE)...\n")
trainer.train()

# ============================================================
# SAVE LORA MODEL
# ============================================================
try:
    # Store original quantization_config from the processor if it exists
    original_processor_quant_config = getattr(processor, 'quantization_config', None)
    if original_processor_quant_config is not None:
        del processor.quantization_config

    # Store original quantization_config from the tokenizer's init_kwargs if it exists
    original_tokenizer_quant_config_in_init_kwargs = None
    if hasattr(processor, 'tokenizer') and hasattr(processor.tokenizer, 'init_kwargs') and 'quantization_config' in processor.tokenizer.init_kwargs:
        original_tokenizer_quant_config_in_init_kwargs = processor.tokenizer.init_kwargs.pop('quantization_config')

    trainer.save_model(OUTPUT_DIR)
    processor.save_pretrained(OUTPUT_DIR)
finally:
    # Restore quantization_config to the processor if it was present
    if original_processor_quant_config is not None:
        processor.quantization_config = original_processor_quant_config

    # Restore quantization_config to the tokenizer's init_kwargs if it was present
    if original_tokenizer_quant_config_in_init_kwargs is not None:
        if hasattr(processor, 'tokenizer') and hasattr(processor.tokenizer, 'init_kwargs'):
            processor.tokenizer.init_kwargs['quantization_config'] = original_tokenizer_quant_config_in_init_kwargs

print(f"\n✅ Training complete. Model saved to: {OUTPUT_DIR}")

/usr/local/lib/python3.12/dist-packages/torch/backends/__init__.py:46: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  self.setter(val)


✅ GPU: Tesla T4


preprocessor_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. 


...

Generating train split: 0 examples [00:00, ? examples/s]


🔥 Starting SFT (15GB GPU SAFE)...



`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss



✅ Training complete. Model saved to: outputs/qwen2_vl_lora_15gb


INFERENCE

In [4]:
!pip install evaluate
!pip install rouge_score
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=2b817a8b2e643be728908c0038f7da50ea54f8263f1667e55d4fdef344a94e92
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.3 MB/s eta 0:00:00


In [5]:
# combined_inference_evaluation_fixed.py
# FIXED: OOM + ROUGE/BERTScore crashes for Qwen2-VL-2B LoRA
# Ready-to-run with memory limits + clean answer extraction

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
import torch
import gc
from pathlib import Path
from PIL import Image
from tqdm import tqdm
from transformers import AutoProcessor, Qwen2VLForConditionalGeneration, BitsAndBytesConfig
from peft import PeftModel
from datasets import load_dataset
from evaluate import load
import warnings
warnings.filterwarnings("ignore")

# ============================================================
# CONFIG (UPDATE THESE PATHS)
# ============================================================
BASE_MODEL_ID = "Qwen/Qwen2-VL-2B-Instruct"
LORA_DIR = "outputs/qwen2_vl_lora_15gb"
EVAL_JSON_PATH = "/content/drive/MyDrive/data/sft/train.json"
IMAGE_ROOT = "/content/drive/MyDrive/data"
MAX_EVAL_SAMPLES = 50  # Start small
MAX_NEW_TOKENS = 64
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE = torch.bfloat16

print(f"Device: {DEVICE}, GPU: {torch.cuda.get_device_name(0) if DEVICE=='cuda' else 'CPU'}")

# ============================================================
# LOAD PROCESSOR WITH MEMORY LIMITS (CRITICAL OOM FIX)
# ============================================================
processor = AutoProcessor.from_pretrained(
    BASE_MODEL_ID,
    min_pixels=256 * 28 * 28,    # ~200k pixels - SAFE
    max_pixels=448 * 28 * 28,    # ~350k pixels - WORKS ON 15GB
    trust_remote_code=True
)
print("✅ Processor loaded with memory limits")

# ============================================================
# LOAD MODEL (4-BIT + LoRA)
# ============================================================
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=DTYPE
)

base_model = Qwen2VLForConditionalGeneration.from_pretrained(
    BASE_MODEL_ID,
    torch_dtype=DTYPE,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=quant_config
)

model = PeftModel.from_pretrained(
    base_model, LORA_DIR,
    torch_dtype=DTYPE,
    device_map="auto",
    is_trainable=False
)
model.eval()
print("✅ Model + LoRA loaded")

# ============================================================
# FIXED INFERENCE (CLEAN ANSWER EXTRACTION)
# ============================================================
def infer(question: str, image_path: str, max_new_tokens: int = MAX_NEW_TOKENS) -> str:
    try:
        # Load & resize image (OOM prevention)
        img = Image.open(image_path).convert("RGB")
        max_side = 896
        if max(img.size) > max_side:
            ratio = max_side / max(img.size)
            new_size = (int(img.width * ratio), int(img.height * ratio))
            img = img.resize(new_size, Image.Resampling.LANCZOS)

        # Messages (match your training format - adjust if needed)
        messages = [{
            "role": "user",
            "content": [
                {"type": "image"},
                {"type": "text", "text": question}
            ]
        }]

        # Process
        text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        inputs = processor(text=[text], images=[img], return_tensors="pt", padding=True).to(DEVICE)

        # Generate
        with torch.no_grad():
            generated_ids = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=0.0,
                do_sample=False,
                eos_token_id=processor.tokenizer.eos_token_id,
                pad_token_id=processor.tokenizer.pad_token_id
            )

        # FIXED: Extract ONLY assistant response (ROUGE/BERTScore fix)
        full_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

        # Method 1: Split by assistant marker
        if "<|im_start|>assistant" in full_text:
            answer = full_text.split("<|im_start|>assistant")[-1].split("<|im_end|>")[0].strip()
        # Method 2: Split by "assistant" (case insensitive)
        elif "assistant" in full_text.lower():
            answer = full_text.lower().split("assistant")[-1].strip()
        # Method 3: Take after last user message
        else:
            answer = full_text.split("user")[-1].strip()

        # Clean vision tokens
        for token in ["<|vision_start|>", "<|vision_end|>", "<|image_pad|>"]:
            answer = answer.replace(token, "")

        answer = answer.strip()

        # Memory cleanup
        del inputs, generated_ids
        torch.cuda.empty_cache()
        gc.collect()

        return answer if answer else "[NO ANSWER]"

    except torch.cuda.OutOfMemoryError:
        torch.cuda.empty_cache()
        gc.collect()
        return "[OOM_ERROR]"
    except Exception as e:
        torch.cuda.empty_cache()
        gc.collect()
        return f"[ERROR: {str(e)[:50]}]"

# ============================================================
# FIXED EVALUATION (HANDLES YOUR DATA FORMAT)
# ============================================================
def evaluate_model(max_samples: int = MAX_EVAL_SAMPLES):
    dataset = load_dataset("json", data_files=EVAL_JSON_PATH, split="train")
    dataset = dataset.shuffle(seed=42).select(range(min(max_samples, len(dataset))))

    predictions = []
    references = []
    valid_count = 0

    print(f"Evaluating {len(dataset)} samples...")

    for i, sample in enumerate(tqdm(dataset, desc="Eval")):
        try:
            messages = sample["messages"]
            user_msg = messages[0]
            assistant_msg = messages[1]

            # FIXED: Handle your exact data format
            user_content = user_msg["content"]
            if isinstance(user_content, list):
                # List format: extract text + image
                question_parts = [item["text"] for item in user_content if item.get("type") == "text"]
                image_items = [item for item in user_content if item.get("type") == "image"]
                question = " ".join(question_parts) if question_parts else ""
                image_rel_path = image_items[0]["image"] if image_items else None
            else:
                # String format
                question = user_content
                image_rel_path = sample.get("image") or sample.get("image_path")

            # Ground truth (FIXED extraction)
            assistant_content = assistant_msg["content"]
            if isinstance(assistant_content, list) and len(assistant_content) > 0:
                ref = assistant_content[0].get("text", "")
            else:
                ref = str(assistant_content)

            if not image_rel_path or not question or not ref:
                continue

            full_image_path = os.path.join(IMAGE_ROOT, image_rel_path)
            if not os.path.exists(full_image_path):
                continue

            # Run inference
            pred = infer(question, full_image_path)

            if "[OOM_ERROR" not in pred and "[ERROR" not in pred and pred.strip():
                predictions.append(pred.strip())
                references.append(ref.strip())
                valid_count += 1
                print(f"[{valid_count}] Q: {question[:50]}... -> P: {pred[:50]}... | R: {ref[:50]}...")

            # Aggressive memory cleanup every 5 samples
            if (i + 1) % 5 == 0:
                torch.cuda.empty_cache()
                gc.collect()

        except Exception as e:
            print(f"Sample {i} error: {e}")
            continue

    print(f"\n✅ Processed {valid_count} valid samples")

    if len(predictions) < 2:
        print("❌ Too few valid predictions for metrics")
        return {"error": "insufficient_valid_samples", "count": valid_count}

    # FIXED: Metrics (pure strings only)
    predictions = [str(p) for p in predictions]
    references = [str(r) for r in references]

    results = {"valid_samples": valid_count, "total_attempted": len(dataset)}

    # ROUGE
    try:
        rouge = load("rouge")
        rouge_results = rouge.compute(predictions=predictions, references=references)
        results.update({
            "ROUGE1": round(rouge_results["rouge1"], 4),
            "ROUGE2": round(rouge_results["rouge2"], 4),
            "ROUGEL": round(rouge_results["rougeL"], 4)
        })
        print("✅ ROUGE computed")
    except Exception as e:
        print(f"❌ ROUGE failed: {e}")
        results["ROUGE"] = "failed"

    # BERTScore
    try:
        bertscore = load("bertscore")
        bert_results = bertscore.compute(predictions=predictions, references=references, lang="en")
        results["BERTScore_F1_mean"] = round(sum(bert_results["f1"]) / len(bert_results["f1"]), 4)
        results["BERTScore_F1_std"] = round(torch.tensor(bert_results["f1"]).std().item(), 4)
        print("✅ BERTScore computed")
    except Exception as e:
        print(f"❌ BERTScore failed: {e}")
        results["BERTScore"] = "failed"

    return results

# ============================================================
# RUN
# ============================================================
if __name__ == "__main__":
    print("\n" + "="*70)
    print("Qwen2-VL-2B LoRA EVALUATION (OOM + Metrics FIXED)")
    print("="*70)

    results = evaluate_model(max_samples=MAX_EVAL_SAMPLES)

    print("\n" + "="*70)
    print("FINAL RESULTS")
    print("="*70)
    for key, value in results.items():
        print(f"{key:20}: {value}")

    print("\n💾 GPU memory:", f"{torch.cuda.memory_allocated()/1e9:.1f}GB / {torch.cuda.memory_reserved()/1e9:.1f}GB reserved")
    torch.cuda.empty_cache()

Device: cuda, GPU: Tesla T4
✅ Processor loaded with memory limits


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/729 [00:00<?, ?it/s]

✅ Model + LoRA loaded

Qwen2-VL-2B LoRA EVALUATION (OOM + Metrics FIXED)
Evaluating 50 samples...


Eval:   2%|▏         | 1/50 [00:10<08:50, 10.82s/it]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/389 [00:00<?, ?it/s]

RobertaModel LOAD REPORT from: roberta-large
Key                             | Status     | 
--------------------------------+------------+-
lm_head.bias                    | UNEXPECTED | 
lm_head.layer_norm.bias         | UNEXPECTED | 
roberta.embeddings.position_ids | UNEXPECTED | 
lm_head.dense.weight            | UNEXPECTED | 
lm_head.dense.bias              | UNEXPECTED | 
lm_head.layer_norm.weight       | UNEXPECTED | 
pooler.dense.bias               | MISSING    | 
pooler.dense.weight             | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


✅ BERTScore computed

FINAL RESULTS
valid_samples       : 50
total_attempted     : 50
ROUGE1              : 0.2303
ROUGE2              : 0.0436
ROUGEL              : 0.1796
BERTScore_F1_mean   : 0.8536
BERTScore_F1_std    : 0.0134

💾 GPU memory: 7.1GB / 7.7GB reserved
